In [ ]:
!wget https://www.dropbox.com/s/n30i1tiwi3s64fg/X-Ray_B.zip?dl=0

In [ ]:
!unzip X-Ray_B.zip?dl=0

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50,VGG16,MobileNet,Xception
from tensorflow.keras.applications.resnet50 import preprocess_input

In [8]:
train_dir = "X-Ray/train/"
valid_dir = "X-Ray/test/"

img_width, img_height = 224, 224

##Xception

In [9]:
conv_base = Xception(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))

conv_base.summary()

83697664/83683744 [==============================] - 1s 0us/step
Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                   

## Features Extraction

In [10]:
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 2048))  
    # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count,))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='binary')
    
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 1046)  
validation_features, validation_labels = extract_features(valid_dir, 466)

Found 1046 images belonging to 2 classes.
Found 466 images belonging to 2 classes.


In [11]:
train_labels

array([0., 1., 0., ..., 1., 0., 1.])

In [12]:
nsamples1, nx1, ny1 ,nz1= train_features.shape
d2_train_dataset = train_features.reshape((nsamples1,nx1*ny1*nz1))

In [13]:
nsamples, nx, ny ,nz= validation_features.shape
d2_test_dataset = validation_features.reshape((nsamples,nx*ny*nz))

### Implementing SVM

In [14]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc=SVC() 
svc.fit(d2_train_dataset,train_labels)

y_pred=svc.predict(d2_test_dataset)

In [ ]:
#print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))
print('Model accuracy score with default hyperparameters: {0:0.4f} : ')

Model accuracy score with default hyperparameters: {0:0.4f} : 


In [16]:
print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))

Model accuracy score with default hyperparameters: 0.9635


In [17]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.96      0.97      0.96       232
         1.0       0.97      0.96      0.96       234

    accuracy                           0.96       466
   macro avg       0.96      0.96      0.96       466
weighted avg       0.96      0.96      0.96       466



In [18]:
confusion_matrix(y_pred,validation_labels)

array([[224,   8],
       [  9, 225]])

### Implementing a Random Forrest

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
newmodel=RandomForestClassifier()

In [21]:
newmodel.fit(d2_train_dataset,train_labels)

RandomForestClassifier()

In [22]:
y_pred=newmodel.predict(d2_test_dataset)
y_pred

array([1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0.,
       1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 1.

In [23]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93       228
         1.0       0.94      0.92      0.93       238

    accuracy                           0.93       466
   macro avg       0.93      0.93      0.93       466
weighted avg       0.93      0.93      0.93       466



In [24]:
confusion_matrix(y_pred,validation_labels)

array([[215,  13],
       [ 18, 220]])

### Implementing a KNN

In [25]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
knn=KNeighborsClassifier(n_neighbors=7)

In [27]:
knn.fit(d2_train_dataset,train_labels)

KNeighborsClassifier(n_neighbors=7)

In [28]:
y_pred_knn=knn.predict(d2_test_dataset)
y_pred_knn

array([1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0.,
       1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 1.

In [29]:
accuracy_score(y_pred_knn,validation_labels)
print(classification_report(y_pred_knn,validation_labels))

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94       232
         1.0       0.94      0.94      0.94       234

    accuracy                           0.94       466
   macro avg       0.94      0.94      0.94       466
weighted avg       0.94      0.94      0.94       466



In [30]:
confusion_matrix(y_pred_knn,validation_labels)

array([[218,  14],
       [ 15, 219]])

## Implementing a Decision Tree

In [31]:
from sklearn.tree import DecisionTreeClassifier

In [32]:
dtc=DecisionTreeClassifier()

In [33]:
dtc.fit(d2_train_dataset,train_labels)

DecisionTreeClassifier()

In [34]:
y_pred_dtc=dtc.predict(d2_test_dataset)

In [35]:
y_pred_dtc

array([1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0.,
       0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 0.

In [36]:
accuracy_score(y_pred_dtc,validation_labels)
print(classification_report(y_pred_dtc,validation_labels))

              precision    recall  f1-score   support

         0.0       0.84      0.83      0.83       237
         1.0       0.82      0.84      0.83       229

    accuracy                           0.83       466
   macro avg       0.83      0.83      0.83       466
weighted avg       0.83      0.83      0.83       466



In [37]:
confusion_matrix(y_pred_dtc,validation_labels)

array([[196,  41],
       [ 37, 192]])

## Implementing Gaussian Naive Bayes

In [38]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(d2_train_dataset,train_labels)

GaussianNB()

In [39]:
y_pred=dtc.predict(d2_test_dataset)

In [40]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.84      0.83      0.83       237
         1.0       0.82      0.84      0.83       229

    accuracy                           0.83       466
   macro avg       0.83      0.83      0.83       466
weighted avg       0.83      0.83      0.83       466



## Implementing AdaBoost

In [41]:
from sklearn.ensemble import AdaBoostClassifier

In [42]:
classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200
)
classifier.fit(d2_train_dataset, train_labels)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=200)

In [43]:
predictions = classifier.predict(d2_test_dataset)

In [44]:
accuracy_score(predictions,validation_labels)
print(classification_report(predictions,validation_labels))

              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       234
         1.0       0.95      0.96      0.95       232

    accuracy                           0.95       466
   macro avg       0.95      0.95      0.95       466
weighted avg       0.95      0.95      0.95       466

